## Deliverable 2. Create a Customer Travel Destinations Map.

In [10]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [11]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,San Vicente,AR,-26.6167,-54.1333,55.67,85,5,1.99,clear sky
1,1,Saint-Philippe,RE,-21.3585,55.7679,73.22,87,62,9.06,light rain
2,2,Punta Arenas,CL,-53.1500,-70.9167,46.11,93,75,8.05,broken clouds
3,3,Taoudenni,ML,22.6783,-3.9836,76.91,22,3,9.40,clear sky
4,4,Bluff,NZ,-46.6000,168.3333,60.80,67,11,5.39,few clouds


In [12]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [13]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                      (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df

,City ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
3,3,Taoudenni,ML,22.6783,-3.9836,76.91,22,3,9.40,clear sky
6,6,Dingle,PH,10.9995,122.6711,80.78,78,72,10.87,broken clouds
7,7,Rikitea,PF,-23.1203,-134.9692,78.55,63,3,11.43,clear sky
11,11,Ixtapa,MX,20.7000,-105.2000,78.22,69,75,8.05,broken clouds
12,12,Caravelas,BR,-17.7125,-39.2481,78.03,62,100,10.83,overcast clouds
...,...,...,...,...,...,...,...,...,...,...
674,674,Mackay,AU,-21.1500,149.2000,79.68,73,40,17.27,scattered clouds
676,676,Road Town,VG,18.4167,-64.6167,78.89,83,40,14.97,scattered clouds
680,680,Taganak,PH,6.0833,118.3000,82.54,73,14,4.23,few clouds
681,681,Bontang,ID,0.1333,117.5000,82.02,78,77,7.70,broken clouds


In [14]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City ID                190
City                   190
Country                190
Lat                    190
Lng                    190
Max Temp               190
Humidity               190
Cloudiness             190
Wind Speed             190
Current Description    190
dtype: int64

In [15]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

In [16]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Taoudenni,ML,76.91,clear sky,22.6783,-3.9836,
6,Dingle,PH,80.78,broken clouds,10.9995,122.6711,
7,Rikitea,PF,78.55,clear sky,-23.1203,-134.9692,
11,Ixtapa,MX,78.22,broken clouds,20.7000,-105.2000,
12,Caravelas,BR,78.03,overcast clouds,-17.7125,-39.2481,
16,Nioro,GM,78.69,clear sky,13.3500,-15.7500,
17,Pitimbu,BR,79.39,broken clouds,-7.4706,-34.8086,
23,Mogadishu,SO,82.36,overcast clouds,2.0371,45.3438,
25,Piacabucu,BR,79.95,few clouds,-10.4056,-36.4344,
33,Goure,NE,79.66,overcast clouds,13.9835,10.2704,


In [17]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.interrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"
    
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"]["name"]
    except (IndexError):
        print("Hotel not found... skipping")       

AttributeError: 'DataFrame' object has no attribute 'interrows'

In [ ]:
# 7. Drop the rows where there is no Hotel Name.
hotels

In [ ]:
# 8a. Create the output File (CSV)

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [ ]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>

"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [18]:
# 11a. Add a marker layer for each city to the map. 
locations = clean_hotel_df[["Lat", "Lng"]]
max_temp = clean_hotel_df["Max Temp"]

fig = gmaps.figure(center=(30.3, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, 
                                 dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

# 11b. Display the figure
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
fig

NameError: name 'clean_hotel_df' is not defined